# Data loading

In [1]:
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import random_split

cifar10_classes = {
    0: "airplane",
    1: "automobile",
    2: "bird",
    3: "cat",
    4: "deer",
    5: "dog",
    6: "frog",
    7: "horse",
    8: "ship",
    9: "truck"
}

cifar10_class_to_idx = {v: k for k, v in cifar10_classes.items()}

label_transform = lambda label: cifar10_classes[label]

dataset = torchvision.datasets.CIFAR10(
    root='/home/azureuser/gautijha37/vision/data',      # Directory where the data will be stored/loaded from
    train=True,        # Load the training set
    download=True,     # Download the dataset if it's not already present
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=(0.5, 0.5, 0.5),
                         std=(0.5, 0.5, 0.5))
    ]),
    target_transform=label_transform   
)

train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size

# Use random_split to safely create subsets
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

In [2]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.backends.cudnn.benchmark = True
device

device(type='cuda')

In [ ]:
from transformers import SiglipModel, SiglipImageProcessor, SiglipTokenizer, SiglipConfig

config = SiglipConfig()
config.vision_config.image_size=32
config.text_config.max_position_embeddings=4 # max length of tokenized classes

config.text_config.num_hidden_layers=3 # using all 12 sees 4% accuracy increase.
config.vision_config.num_hidden_layers=3

model = SiglipModel(config).to(device)

img_processor = SiglipImageProcessor(do_resize=False, do_rescale=False) # transforms.ToTensor() already scales input img to [-1, 1]
tokenizer = SiglipTokenizer.from_pretrained("google/siglip-base-patch16-224")


/home/azureuser/gautijha37/vision/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
sum(p.numel() for p in model.parameters()) // 1e6

75.0

# Training

In [5]:
from tqdm import tqdm
def accuracy(model, tokenizer, data_loader):
    correct = 0
    all_text_inputs = list(cifar10_classes.values())
    text_inputs = tokenizer(text=all_text_inputs, padding=True, return_tensors="pt")
    text_inputs['input_ids']=text_inputs['input_ids'].to(device)

    for img_inputs, _, _, correct_labels in tqdm(data_loader):
        img_inputs['pixel_values']=img_inputs['pixel_values'].to(device)
        
        output = model(**text_inputs, **img_inputs)
        predicted_indices = output.logits_per_image.argmax(dim=1)
        predicted_labels = [cifar10_classes[k.item()] for k in predicted_indices]
        
        correct += sum(p == c for p, c in zip(predicted_labels, list(correct_labels)))

    total = len(data_loader) * data_loader.batch_size
    return 100 * correct/total

In [ ]:
from tqdm import tqdm
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import CosineAnnealingLR
import wandb

num_epochs = 50
batch_size = 64
lr = 1e-4

wandb.init(project="siglip-cifar10", config={
    "num_epochs": num_epochs,
    "batch_size": batch_size,
    "learning_rate": lr,
    "model": "SigLIP",
    "dataset": "CIFAR10",
    "num_hidden_layers": config.text_config.num_hidden_layers,
    "image_size": config.vision_config.image_size,
    "max_position_embeddings": config.text_config.max_position_embeddings
})

# Dataloader
def collate_fn(batch):
    images, labels = zip(*batch)
    img_inputs = img_processor(images=images, return_tensors="pt")
    text_inputs = tokenizer(text = labels, padding=True, return_tensors="pt")
    
    labels_ids = torch.tensor([cifar10_class_to_idx[label] for label in labels])
    batch_mask = (labels_ids.unsqueeze(1) == labels_ids.unsqueeze(0)).float()
    batch_mask = 2 * batch_mask - 1
    
    return img_inputs, text_inputs, batch_mask, labels

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=16, collate_fn=collate_fn)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True, num_workers=16, collate_fn=collate_fn)

# Optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)

scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
    optimizer,
    T_max=num_epochs,
    eta_min=1e-6
)

# loss calculation
def calculate_loss(model, batch):
    img_inputs, text_inputs, batch_mask, _ = batch
    img_inputs['pixel_values']=img_inputs['pixel_values'].to(device)
    text_inputs['input_ids']=text_inputs['input_ids'].to(device)
    batch_mask=batch_mask.to(device)
    
    # loss calculation
    output = model(**text_inputs, **img_inputs)
    loglik = torch.nn.functional.logsigmoid(batch_mask * output.logits_per_text)
    nll = -torch.sum(loglik, dim=-1)
    loss = nll.mean()
    
    return loss

# Training loop
for epoch in tqdm(range(num_epochs)):
    # Training
    model.train()
    for batch_id, batch in enumerate(train_loader):
        loss = calculate_loss(model, batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        wandb.log({
            "batch_loss": loss.item(),
            "learning_rate": optimizer.param_groups[0]['lr']
        })
    
    scheduler.step()
    
    # Validation
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for batch in val_loader:
            val_loss += calculate_loss(model, batch)
    
    val_loss /= len(val_loader)
    val_accuracy = accuracy(model, tokenizer, val_loader)
    print(f"Epoch [{epoch+1}/{num_epochs}], train_loss: {loss:.4f}, val_loss: {val_loss:.4f}, val_accuracy: {val_accuracy:.4f}")
    wandb.log({
        "epoch_train_loss": loss,
        "epoch_val_loss": val_loss,
        "epoch_val_accuracy": val_accuracy
    })
    

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: gauti-jha37 (gauti-jha37-personal) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


100%|██████████| 157/157 [00:01<00:00, 104.55it/s]


Epoch [1/50], train_loss: 26.0088, val_loss: 26.1353, val_accuracy: 9.8627


  4%|▍         | 2/50 [00:30<12:13, 15.28s/it]

Epoch [2/50], train_loss: 25.1383, val_loss: 25.2227, val_accuracy: 11.0768


...

Epoch [48/50], train_loss: 9.6865, val_loss: 18.4388, val_accuracy: 42.0084


 98%|█████████▊| 49/50 [12:32<00:15, 15.27s/it]

Epoch [49/50], train_loss: 10.5721, val_loss: 18.4961, val_accuracy: 41.8889


100%|██████████| 50/50 [12:47<00:00, 15.36s/it]

Epoch [50/50], train_loss: 11.3788, val_loss: 18.5073, val_accuracy: 41.8690


# Testing

In [8]:
from safetensors.torch import save_file
state_dict = model.state_dict()
save_file(state_dict, "/home/azureuser/gautijha37/vision/siglip.safetensors6")

In [7]:
test_dataset = torchvision.datasets.CIFAR10(
    root='/home/azureuser/gautijha37/vision/data',
    train=False,        # Load the test set
    download=True,
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=(0.5, 0.5, 0.5),
                         std=(0.5, 0.5, 0.5))
    ]),
    target_transform=label_transform
)

test_loader = DataLoader(test_dataset, batch_size=batch_size, num_workers=16, shuffle=True, collate_fn=collate_fn)
print(accuracy(model, tokenizer, test_loader))

100%|██████████| 157/157 [00:01<00:00, 105.65it/s]

41.590366242038215
